Libraries

In [1]:
import psycopg2
from psycopg2.extensions import AsIs

import glob
import pandas as pd
import io

import time

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

import warnings
warnings.filterwarnings('ignore')

from config import db_info

DB Connection

In [2]:
schema_name = 'sc'

conn = psycopg2.connect(**db_info)
conn.autocommit = True

curs = conn.cursor()
curs.execute(f'set search_path to {schema_name}')

SureChembl Files

In [3]:
data_folder = './sure_chembl_data/'
files = glob.glob(data_folder + '*.txt.gz')

colnames = ['id', 'smiles', 'inchi_key', 'corpus_freq', 'patent_id', \
            'pub_date', 'field_type', 'field_freq']

compound = 'compound'
cmp_cols = ['id', 'smiles']

patent = 'patent'
pat_cols = ['patent_id', 'pub_date']

field_freq = 'field_freq'
fld_cols = ['id', 'patent_id', 'field_type', 'field_freq']

Populating DB

In [11]:
for f_idx, f in enumerate(files):
    ###
    # read one file
    df = pd.read_csv(f, header=None, sep='\t')
    df.columns = colnames

    # filter the file
    df = df.drop(['inchi_key', 'corpus_freq'], axis=1)
    df['id'] = df.id.apply(lambda x: x.split('SCHEMBL')[1]).astype(int)
    df['patent_id'] = df.patent_id.apply(lambda x: ''.join(x.split('-')))
    df = df[df.patent_id.str[0:2] == 'US']
    ###

    ###
    container = dict()
    for tbl, cols in zip([compound, patent, field_freq], [cmp_cols, pat_cols, fld_cols]):
        container[tbl] = df[cols].drop_duplicates(cols[:-1])
    df = None
    ###

    ###
    # upserting compounds
    start = time.time()
    invalid_compounds = []
    i_idx = 0
    for id, smi in zip(container[compound].id, container[compound].smiles):
        if i_idx % 50000 == 0 and i_idx > 0:
            print(f'Processed {i_idx} compounds out of {len(container[compound])}.')
        
        m = Chem.MolFromSmiles(smi)
        if not m: 
            invalid_compounds.append(id)
            continue
        can_smi = Chem.MolToSmiles(m, canonical=True)
        if not can_smi: 
            invalid_compounds.append(id)
            continue
        b_mfp = DataStructs.BitVectToBinaryText( AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) )

        try:
            curs.execute('insert into %(tbl)s values (%(id)s, %(smi)s::mol, bfp_from_binary_text(%(mfp)s)) \
                on conflict (id) do update set smiles = excluded.smiles, mfp = excluded.mfp',
                {'tbl': AsIs(compound), 'id': id, 'smi': can_smi, 'mfp': b_mfp})
            
        except Exception as e:
            print(e)
            invalid_compounds.append(id)
        
        m = can_smi = b_mfp = None
        i_idx += 1
    
    del container[compound]

    if len(invalid_compounds) >= 1:
        container[field_freq] = container[field_freq][ ~(container[field_freq].id.isin(invalid_compounds)) ]
        container[patent] = container[patent][ (container[patent].patent_id.isin( container[field_freq].patent_id.unique().tolist() )) ]
    invalid_compounds = None
    
    print(f'Compounds took {time.time() - start} seconds.')
    ###
    
    ###
    # upserting patents
    start = time.time()
    curs.execute(f'create table tmp_{patent} (like {patent} including defaults)')

    buf = io.StringIO(container[patent].to_csv(index=False, header=True, sep=','))
    del container[patent]
    curs.copy_expert(sql=f"copy tmp_{patent} (num, pub_date) from stdin with csv header delimiter as ','", file=buf)
    buf = None

    curs.execute(f'insert into {patent} select * from tmp_{patent} on conflict (num) \
                 do update set pub_date = excluded.pub_date returning num, id')
    
    # need the patent map!
    patent_map = {k: v for k, v in curs.fetchall()}

    curs.execute(f'drop table tmp_{patent}')
    container[field_freq]['patent_id'] = container[field_freq].patent_id.map(patent_map)
    patent_map = None

    print(f'Patents took {time.time() - start} seconds.')
    ###

    ###
    # upserting field frequencies
    start = time.time()

    curs.execute(f'create table tmp_{field_freq} (like {field_freq} including defaults)')

    buf = io.StringIO(container[field_freq].to_csv(index=False, header=True, sep=','))
    del container[field_freq]
    curs.copy_expert(sql=f"copy tmp_{field_freq} from stdin with csv header delimiter as ','", file=buf)
    buf = None

    curs.execute(f'insert into {field_freq} select * from tmp_{field_freq} on conflict (compound_id, patent_id, field_id) \
                 do update set freq = excluded.freq')
    curs.execute(f'drop table tmp_{field_freq}')

    print(f'Field freqs took {time.time() - start} seconds.')
    ###

    print(f'Processed file number {f_idx + 1} out of {len(files)}.')

Processed 50000 compounds out of 664622.
could not create molecule from SMILES 'CCCCCC(=O)[O-].C[Mg+2]'
LINE 1: insert into compound values (151105, 'CCCCCC(=O)[O-].C[Mg+2]...
                                             ^

could not create molecule from SMILES 'CC1(C)CCCC(C)(C)[N-]1.[Li]Cl.[Mg+2]Cl'
LINE 1: insert into compound values (9894115, 'CC1(C)CCCC(C)(C)[N-]1...
                                              ^

could not create molecule from SMILES 'CCCC[Mg+2].CC[O-]'
LINE 1: insert into compound values (1563033, 'CCCC[Mg+2].CC[O-]'::m...
                                              ^

Processed 100000 compounds out of 664622.
could not create molecule from SMILES 'CCCC[O-].CC[Mg+2]'
LINE 1: insert into compound values (1988209, 'CCCC[O-].CC[Mg+2]'::m...
                                              ^

could not create molecule from SMILES 'CC(C)C[O-].CCC[Mg+2]'
LINE 1: insert into compound values (1988608, 'CC(C)C[O-].CCC[Mg+2]'...
                                            

Creating Indices

In [12]:
curs.execute('create index smi_idx on compound using gist(smiles)')
curs.execute('create index mfp_idx on compound using gist(mfp)')

Testing Similarity and Substructure Searching

In [6]:
s = 'CC[C@]1(c2ccccc2)C(=O)NC(=O)N(C)C1=O'

curs.execute('select c.id, mol_to_smiles(c.smiles), p.num from compound c, patent p, field_freq f \
             where c.id = f.compound_id and p.id = f.patent_id \
             and c.smiles@>%s', (s, ))

r = curs.fetchall()

In [8]:
r[0]

(1255360,
 'CCCCOCC(CN1C(=O)N(CC(COCCCC)OC(N)=O)C(=O)C(CC)(c2ccccc2)C1=O)OC(N)=O',
 'US20150004244A1')

In [9]:
s = 'CC[C@]1(c2ccccc2)C(=O)NC(=O)N(C)C1=O'
m = Chem.MolFromSmiles(s)
b_mfp = DataStructs.BitVectToBinaryText( AllChem.GetMorganFingerprintAsBitVect(m, 2, nBits=1024) )

curs.execute('set rdkit.tanimoto_threshold=0.95')
curs.execute('select c.id, mol_to_smiles(c.smiles), p.num from compound c, patent p, field_freq f \
             where c.id = f.compound_id and p.id = f.patent_id \
             and c.mfp%%bfp_from_binary_text(%s)', (b_mfp, ))

r = curs.fetchall()

In [12]:
r[0]

(35625, 'CC[C@]1(c2ccccc2)C(=O)NC(=O)N(C)C1=O', 'US8952008B2')

Close DB Connection

In [13]:
curs.close()
conn.close()